<a href="https://colab.research.google.com/github/Lexi-Zhou/stats201-project-zzz/blob/main/Code/W3_3_Gender_Check_Finalized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd

pd.set_option("display.max_colwidth", 160)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_3/"
FILE_NAME = "11_RMP_prof_gender_preprocessed.csv"

FILE_PATH = os.path.join(DATA_DIR, FILE_NAME)

df = pd.read_csv(FILE_PATH)

print("Dataset shape:", df.shape)
df.head()


Dataset shape: (19685, 18)


,professor_name,school_name,department_name_clean,stu_tags,student_star,comments,tag_list,comment_gender_signal,prof_gender_label,star_rating,star_rating_category,student_star_category,comments_raw,comments_basic,comments_expanded,comments_tokens,comments_final,n_tokens_final
0,Lynn Ketter,Northwest Florida State College,Education,RESPECTED CARING GIVES GOOD FEEDBACK,5.0,"Wonderful teacher. Basically all work is done in class. The grading is very clear, plus she gives feedback on every assignment, and lets you turn it in agai...","['RESPECTED', 'CARING', 'GIVES GOOD FEEDBACK']",female,female,3.7,good,good,"Wonderful teacher. Basically all work is done in class. The grading is very clear, plus she gives feedback on every assignment, and lets you turn it in agai...","Wonderful teacher. Basically all work is done in class. The grading is very clear, plus she gives feedback on every assignment, and lets you turn it in agai...","wonderful teacher. basically all work is done in class. the grading is very clear, plus she gives feedback on every assignment, and lets you turn it in agai...","['wonderful', 'teacher', 'basically', 'work', 'class', 'grading', 'very', 'clear', 'plus', 'gives', 'feedback', 'assignment', 'lets', 'turn', 'points']",wonderful teacher basically work class grading very clear plus gives feedback assignment lets turn points,15
1,Lynn Ketter,Northwest Florida State College,Education,PARTICIPATION MATTERS,2.0,Discussion boards are mandatory. Use book for tests. Personally found her a little rude via e-mail and D2L. Seems very informative with some aspects of the ...,['PARTICIPATION MATTERS'],female,female,3.7,good,poor,Discussion boards are mandatory. Use book for tests. Personally found her a little rude via e-mail and D2L. Seems very informative with some aspects of the ...,Discussion boards are mandatory. Use book for tests. Personally found her a little rude via e-mail and D2L. Seems very informative with some aspects of the ...,discussion boards are mandatory. use book for tests. personally found her a little rude via e-mail and d2l. seems very informative with some aspects of the ...,"['discussion', 'boards', 'mandatory', 'use', 'book', 'tests', 'personally', 'little', 'rude', 'e', 'mail', 'd', 'l', 'very', 'informative', 'aspects', 'clas...",discussion boards mandatory use book tests personally little rude e mail d l very informative aspects class changes things d l content so check stuff day,26
2,Lynn Ketter,Northwest Florida State College,Education,GIVES GOOD FEEDBACK LOTS OF HOMEWORK PARTICIPATION MATTERS,3.5,"Honestly, I didn\'t learn anything in this class but Ketter is amazing at giving great feedback and allowing resubmissions. Textbook is absolutely pointless...","['GIVES GOOD FEEDBACK', 'LOTS OF HOMEWORK', 'PARTICIPATION MATTERS']",unknown,female,3.7,good,good,"Honestly, I didn\'t learn anything in this class but Ketter is amazing at giving great feedback and allowing resubmissions. Textbook is absolutely pointless...","Honestly, I didn't learn anything in this class but Ketter is amazing at giving great feedback and allowing resubmissions. Textbook is absolutely pointless,...","honestly, i did not learn anything in this class but ketter is amazing at giving great feedback and allowing resubmissions. textbook is absolutely pointless...","['honestly', 'did', 'not', 'NEG_learn', 'class', 'ketter', 'amazing', 'giving', 'great', 'feedback', 'allowing', 'resubmissions', 'textbook', 'absolutely', ...",honestly did not NEG_learn class ketter amazing giving great feedback allowing resubmissions textbook absolutely pointless,15
3,Lynn Ketter,Northwest Florida State College,Education,GIVES GOOD FEEDBACK LOTS OF HOMEWORK SKIP CLASS? YOU WON\'T PASS.,4.0,She comes off a little cold at first but she\'s actually really nice. She has a lot of passion for teaching future educators. No tests in her class but quit...,"['GIVES GOOD FEEDBACK', 'LOTS OF HOMEWORK', ""SKIP CLASS? YOU WON\\'T PASS.""]",female,female,3.7,goo

In [ ]:

SSA_FILE = os.path.join(DATA_DIR, "yob2015.txt")
ssa_df = pd.read_csv(SSA_FILE, names=['first_name', 'gender', 'count'])

ssa_df['first_name'] = ssa_df['first_name'].str.lower()

grouped = ssa_df.groupby(['first_name', 'gender'])['count'].sum().unstack(fill_value=0)

def choose_gender(row):
    if row['F'] > row['M']:
        return 'female'
    elif row['M'] > row['F']:
        return 'male'
    else:
        return 'unknown'

name_gender_dict = grouped.apply(choose_gender, axis=1).to_dict()

df['professor_first_name'] = df['professor_name'].str.split().str[0].str.lower()

df['professor_gender'] = df['professor_first_name'].map(name_gender_dict).fillna('unknown')

unknown_count = (df['professor_gender'] == 'unknown').sum()
print(f"Number of professors with unknown gender: {unknown_count}")

Number of professors with unknown gender: 2456


In [ ]:
NEW_FILE_NAME = "13_RMP_prof_gender_ssaadded.csv"
NEW_FILE_PATH = os.path.join(DATA_DIR, NEW_FILE_NAME)

# Create a copy of the DataFrame to modify without altering the original 'df'
df_to_save = df.copy()

# Rename the 'professor_gender' column to 'prof_gender_ssa'
df_to_save = df_to_save.rename(columns={'professor_gender': 'prof_gender_ssa'})

# Drop the 'professor_first_name' column as it's not needed in the output file
if 'professor_first_name' in df_to_save.columns:
    df_to_save = df_to_save.drop(columns=['professor_first_name'])

# Save the modified DataFrame to CSV
df_to_save.to_csv(NEW_FILE_PATH, index=False)

print(f"DataFrame saved to: {NEW_FILE_PATH}")

DataFrame saved to: /content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_3/13_RMP_prof_gender_ssaadded.csv


In [ ]:
# Rename 'professor_gender' to 'prof_gender_ssa' in the main df to match the column used in crosscheck_gender
# This ensures the column exists for the crosscheck function
if 'professor_gender' in df.columns:
    df = df.rename(columns={'professor_gender': 'prof_gender_ssa'})

# Define a function to compare the two gender columns
def crosscheck_gender(row):
    label = row['prof_gender_label']
    ssa = row['prof_gender_ssa']

    # New rule: If prof_gender_label is 'conflict', then prof_gender_crosschecked is 'conflict'
    if label == 'conflict':
        return 'conflict'

    # Rule 4: Both are unknown
    if label == 'unknown' and ssa == 'unknown':
        return 'unknown'
    # Rule 3: label is unknown, ssa shows gender (user wants 'unknown' here)
    elif label == 'unknown' and ssa != 'unknown':
        return 'unknown'
    # Rule 2: label shows gender, ssa is unknown
    elif label != 'unknown' and ssa == 'unknown':
        return label
    # Rule 1: Both show gender and are consistent (implicitly handled if not 'unknown')
    elif label == ssa:
        return label
    # Rule 5: Otherwise (label and ssa are different specific genders), return label
    else:
        return label

# Apply the function to create the new 'prof_gender_crosschecked' column
df['prof_gender_crosschecked'] = df.apply(crosscheck_gender, axis=1)

# Display the relevant columns to show the result
display(df[['prof_gender_label', 'prof_gender_ssa', 'prof_gender_crosschecked']].head())

# --- Code to save the DataFrame with the new column ---
NEW_FILE_NAME = "13_RMP_prof_gender_ssaadded.csv"
NEW_FILE_PATH = os.path.join(DATA_DIR, NEW_FILE_NAME)

# Create a copy of the DataFrame to modify without altering the original 'df'
df_to_save = df.copy()

# Drop the 'professor_first_name' column as it's not needed in the output file
if 'professor_first_name' in df_to_save.columns:
    df_to_save = df_to_save.drop(columns=['professor_first_name'])

# Save the modified DataFrame to CSV
df_to_save.to_csv(NEW_FILE_PATH, index=False)

print(f"DataFrame saved to: {NEW_FILE_PATH}")

,prof_gender_label,prof_gender_ssa,prof_gender_crosschecked
0,female,female,female
1,female,female,female
2,female,female,female
3,female,female,female
4,female,female,female


DataFrame saved to: /content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_3/13_RMP_prof_gender_ssaadded.csv


In [ ]:
conflict_count = (df['prof_gender_crosschecked'] == 'conflict').sum()
unknown_count  = (df['prof_gender_crosschecked'] == 'unknown').sum()

print(f"Number of comments with conflict gender: {conflict_count}")
print(f"Number of comments with unknown gender: {unknown_count}")

Number of comments with conflict gender: 220
Number of comments with unknown gender: 328


In [ ]:
# Define the aggregation function to determine the final gender status for each unique professor
def aggregate_final_gender(series):
    # If any of the individual entries for a professor is 'conflict', the aggregated status is 'conflict'
    if 'conflict' in series.values:
        return 'conflict'

    # Filter out 'unknown' values to check for consistent known genders
    unique_non_unknown = series[series != 'unknown'].unique()

    # If there's more than one unique non-'unknown' gender (e.g., 'female' and 'male'), it's a conflict
    if len(unique_non_unknown) > 1:
        return 'conflict'
    # If there's exactly one unique non-'unknown' gender (e.g., all 'female'), return that gender
    elif len(unique_non_unknown) == 1:
        return unique_non_unknown[0]
    # If all entries were 'unknown', or no known genders were found, return 'unknown'
    else:
        return 'unknown'

# Group by professor identifiers and apply the aggregation function
# This results in a DataFrame where each row is a unique professor (by name, department, school)
# with their aggregated cross-checked gender status.
df_grouped_professors = df.groupby(['professor_name', 'department_name_clean', 'school_name'])['prof_gender_crosschecked'].apply(aggregate_final_gender).reset_index()

# Filter the grouped DataFrame to keep rows where the aggregated gender status is 'conflict' or 'unknown'
df_conflicted_professors = df_grouped_professors[df_grouped_professors['prof_gender_crosschecked'].isin(['conflict', 'unknown'])]

# Drop the 'prof_gender_crosschecked' column from the filtered DataFrame as requested
df_conflicted_professors = df_conflicted_professors.drop(columns=['prof_gender_crosschecked'])

# Drop department_name_clean and school_name columns as requested
if 'department_name_clean' in df_conflicted_professors.columns:
    df_conflicted_professors = df_conflicted_professors.drop(columns=['department_name_clean'])
if 'school_name' in df_conflicted_professors.columns:
    df_conflicted_professors = df_conflicted_professors.drop(columns=['school_name'])

# Define the new file name and path for the conflicted professors data
NEW_FILE_NAME_CONFLICTED = "14_RMP_prof_gender_conflicted.csv"
NEW_FILE_PATH_CONFLICTED = os.path.join(DATA_DIR, NEW_FILE_NAME_CONFLICTED)

# Save the resulting DataFrame to a new CSV file
df_conflicted_professors.to_csv(NEW_FILE_PATH_CONFLICTED, index=False)

print(f"Conflicted professors data saved to: {NEW_FILE_PATH_CONFLICTED}")
print(f"Number of conflicted or unknown professors: {len(df_conflicted_professors)}")

Conflicted professors data saved to: /content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_3/14_RMP_prof_gender_conflicted.csv
Number of conflicted or unknown professors: 252


In [ ]:
import pandas as pd
import os

# Define file paths for the new input and output files
PREDICTED_GENDER_FILE = os.path.join(DATA_DIR, "15_namsor_genderize-full-name.csv")
FINALIZED_GENDER_FILE = os.path.join(DATA_DIR, "16_RMP_gender_finalized.csv")

# Load the predicted genders DataFrame
df_predicted_gender = pd.read_csv(PREDICTED_GENDER_FILE)

# Ensure 'professor_name' is clean for merging (e.g., lowercased, stripped if necessary)
# Assuming professor_name in both dataframes are consistent for merging
df_predicted_gender['professor_name'] = df_predicted_gender['professor_name'].str.strip().str.lower()

# Ensure the main df's professor_name is also clean if not already
df['professor_name_cleaned'] = df['professor_name'].str.strip().str.lower()

# Merge the predicted genders into the main DataFrame
# Use a left merge to keep all rows from df and add 'likelyGender' where professor_name matches
df_final = pd.merge(
    df,
    df_predicted_gender[['professor_name', 'likelyGender']],
    left_on='professor_name_cleaned',
    right_on='professor_name',
    how='left',
    suffixes=('', '_predicted_merge') # Add suffix to avoid column name clash for 'professor_name'
)

# Rename the merged 'likelyGender' column to 'prof_likely_gender'
df_final = df_final.rename(columns={'likelyGender': 'prof_likely_gender'})

# Drop the temporary cleaned professor name column and the redundant professor_name_predicted_merge column
df_final = df_final.drop(columns=['professor_name_cleaned', 'professor_name_predicted_merge'], errors='ignore')

# For professors not found in the predicted file, 'prof_likely_gender' will be NaN. Fill with 'unknown'
df_final['prof_likely_gender'] = df_final['prof_likely_gender'].fillna('unknown')

# Save the finalized DataFrame to a new CSV file
df_final.to_csv(FINALIZED_GENDER_FILE, index=False)

print(f"Finalized DataFrame saved to: {FINALIZED_GENDER_FILE}")
print("First 5 rows of the finalized DataFrame with prof_likely_gender column:")
display(df_final[['professor_name', 'prof_likely_gender']].head())

Finalized DataFrame saved to: /content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_3/16_RMP_gender_finalized.csv
First 5 rows of the finalized DataFrame with prof_likely_gender column:


,professor_name,prof_likely_gender
0,Lynn Ketter,unknown
1,Lynn Ketter,unknown
2,Lynn Ketter,unknown
3,Lynn Ketter,unknown
4,Lynn Ketter,unknown


In [ ]:
# Define the function to determine the final gender based on the specified rules
def determine_final_gender(row):
    crosschecked = row['prof_gender_crosschecked']
    label = row['prof_gender_label']
    ssa = row['prof_gender_ssa']
    likely = row['prof_likely_gender']

    # Rule 1: If prof_gender_crosschecked is 'unknown'
    if crosschecked == 'unknown':
        return likely

    # Rule 2: If prof_gender_crosschecked shows a specific gender ('female' or 'male')
    elif crosschecked in ['female', 'male']:
        return crosschecked

    # Rule 3: If prof_gender_crosschecked is 'conflict' (UPDATED LOGIC AGAIN)
    elif crosschecked == 'conflict':
        # Condition 1: If SSA is unknown, rely on likely gender (could be unknown or known)
        if ssa == 'unknown':
            return likely
        # Now we know ssa is a specific gender (female/male)
        else:
            # Check if likely gender is also a specific gender
            if likely != 'unknown':
                # Both ssa and likely are known (female/male)
                if ssa == likely:
                    return ssa # Both known and consistent
                else: # ssa != likely, both known but inconsistent -> manual
                    return 'manual' # User's explicit manual condition

# Apply the function to create the new 'prof_gender_finalized' column
df_final['prof_gender_finalized'] = df_final.apply(determine_final_gender, axis=1)

# Save the finalized DataFrame to a new CSV file (overwriting if it exists)
FINALIZED_GENDER_FILE = os.path.join(DATA_DIR, "16_RMP_gender_finalized.csv")
df_final.to_csv(FINALIZED_GENDER_FILE, index=False)

print(f"Updated DataFrame with 'prof_gender_finalized' saved to: {FINALIZED_GENDER_FILE}")

# Filter for 'manual' entries and display count and content
manual_gender_rows = df_final[df_final['prof_gender_finalized'] == 'manual']
print(f"\nNumber of rows with 'manual' gender: {len(manual_gender_rows)}")
print("First 5 rows with 'manual' gender:")
display(manual_gender_rows[['professor_name', 'prof_gender_label', 'prof_gender_ssa', 'prof_gender_crosschecked', 'prof_likely_gender', 'prof_gender_finalized']].head())

# Filter for 'conflict' entries and display count and content
conflict_gender_rows = df_final[df_final['prof_gender_finalized'] == 'conflict']
print(f"\nNumber of rows with 'conflict' gender: {len(conflict_gender_rows)}")
print("First 5 rows with 'conflict' gender:")
display(conflict_gender_rows[['professor_name', 'prof_gender_label', 'prof_gender_ssa', 'prof_gender_crosschecked', 'prof_likely_gender', 'prof_gender_finalized']].head())

Updated DataFrame with 'prof_gender_finalized' saved to: /content/drive/MyDrive/Colab Notebooks/stats201_final_project/Week_3/16_RMP_gender_finalized.csv

Number of rows with 'manual' gender: 0
First 5 rows with 'manual' gender:


,professor_name,prof_gender_label,prof_gender_ssa,prof_gender_crosschecked,prof_likely_gender,prof_gender_finalized



Number of rows with 'conflict' gender: 0
First 5 rows with 'conflict' gender:


,professor_name,prof_gender_label,prof_gender_ssa,prof_gender_crosschecked,prof_likely_gender,prof_gender_finalized
